### Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений


In [293]:
import pandas as pd
import re

In [265]:
data_traffic_source = pd.read_csv(r'visit_log.csv', sep=';')

In [266]:
def traffic_source_f(region, traffic_source):
    if traffic_source == 'yandex' or traffic_source == 'google':
        return 'organic'
    elif (traffic_source == 'paid' or traffic_source == 'email') and region == 'Russia':
        return 'ad'
    elif (traffic_source == 'paid' or traffic_source == 'email') and region != 'Russia':
        return 'other'
    return traffic_source

In [267]:
data_traffic_source['source_type'] = list(map(traffic_source_f,data_traffic_source['region'], data_traffic_source['traffic_source']))

In [268]:
data_traffic_source.tail()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad
18937,1550094323,b3b634f824,https://host.ru/39fa884393666d45fc,Russia,7d27a58fb8,email,ad


### Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном


In [218]:
data_news = pd.read_csv(r'URLs.txt')

In [219]:
data_news = data_news[data_news['url'].str.contains(r'/\d{8}-')]

In [220]:
data_news.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [221]:
data_news['url'][3]

'/politics/36188461-s-marta-zhizn-rossiyan-suschestvenno-izmenitsya-iz-za-novyh-zakonov/'

### Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
data_ratings = pd.read_csv(r'.\ml-latest-small\ratings.csv')

In [187]:
def user_lifetime(timestamp_f):
    return timestamp_f.max() - timestamp_f.min()

In [190]:
data_ratings_user = data_ratings.pivot_table(index = ['userId'],  values = ['movieId','timestamp'], 
                                             aggfunc = {'movieId' : 'count',
                                                        'timestamp' : user_lifetime })
data_ratings_user.query('movieId > 100', inplace = True)


In [191]:
data_ratings_user.head()

,movieId,timestamp
userId,,
4,204,203560
8,116,85187
15,1700,471393496
17,363,8053
19,423,5282


### Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента.

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

    rzd - железнодорожные перевозки
    auto - автомобильные перевозки
    air - воздушные перевозки
    client_base - адреса клиентов

In [274]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [294]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [295]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [296]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base    

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [312]:
client_no_address = air.merge((rzd.merge(auto, how='outer')), how='outer')
client_no_address

,client_id,air_revenue,rzd_revenue,auto_revenue
0,115,81.0,981.0,912.0
1,116,4.0,NaN,4834.0
2,117,13.0,NaN,98.0
3,118,173.0,NaN,NaN
4,111,NaN,1093.0,NaN
5,112,NaN,2810.0,NaN
6,113,NaN,10283.0,57483.0
7,114,NaN,5774.0,83.0


In [309]:
client_address = client_base.merge(client_no_address, how='outer')
client_address

,client_id,address,air_revenue,rzd_revenue,auto_revenue
0,111,Комсомольская 4,NaN,1093.0,NaN
1,112,Энтузиастов 8а,NaN,2810.0,NaN
2,113,Левобережная 1а,NaN,10283.0,57483.0
3,114,Мира 14,NaN,5774.0,83.0
4,115,ЗЖБИиДК 1,81.0,981.0,912.0
5,116,Строителей 18,4.0,NaN,4834.0
6,117,Панфиловская 33,13.0,NaN,98.0
7,118,Мастеркова 4,173.0,NaN,NaN
